### workbook - anlyze 1 tiff image using 2d-pris

In [2]:
import sys
sys.path.append("/u/home/x/xiyuyi/bin")
sys.path.append("../PyPRIS")
sys.path.append('/Users/yi10/PyPRIS')
from PyPRIS import *
import copy
import os

In [10]:
# Define the paths and the file names
dpath='/Users/yi10/Desktop/Research/Projects/P-PRIS/PyPRIS/2d-spt-demo'
tiff_path=dpath+'/frame700.tif'
psf_path=dpath+'/psf.tif'
tpfd='./ticket_pack'
os.mkdir(tpfd)

#### prepare a ticket for pris reconstruction

In [11]:
# get a ticket of 'SinglePlaneTicket' class.
ticket = SinglePlaneTicket()
frameN = 0;
# define basic properties of the ticket that we do not want to explore in this parameter optimization process.
ticket.datapath = dpath # this the folder where the data is stored.
ticket.observation_channel_n = 'SinglePlane'
ticket.plane1_path = tiff_path
ticket.psf_path = psf_path
ticket.psf_norm_factor = 1 # The PSF normalization factor is a number that 
                              # normalizes the total energy of a PSF to be 1
                              # But experimental calibration data, the PSF matrix is often noisy, 
                              # so an estimate would be enough.
            # when the background is too strong, we change this to a smaller number.. forces the scale of the signal measures to be higher

# specification of the psf stack
ticket.psfz0 = 4  # the count of the center plane, define it as z=0 in the psf coordinates system.
ticket.observer_edge_padding = False # switch to True when PSF has high bg component.

# configure the initial candidate pool of this pris ticket
ticket.init_candidates_intervals = list([1, 1, 1])

ticket.init_ax0_range = list([0, 1])
ticket.init_ax1_range = list([0, 128])
ticket.init_ax2_range = list([0, 128])

# debug configurations"
ticket.observer_debugger = False
ticket.tobserver_edge_padding = True

# output settings"
ticket.ticket_pack_folder = tpfd

# linbreg configurations"
ticket.linbreg_alpha = LinBreg("X")
ticket.linbreg_alpha.debug = True
ticket.linbreg_alpha.deep_debug = False

# other parameters
ticket.PRIS_iter_end = 5
ticket.expansion = False

# initial configuration for linbreg object.
ticket.linbreg_alpha.maxit = 3000
ticket.linbreg_alpha.maxit_delta = 1000 # additional threshold to the maxit after each pris iteration.
ticket.linbreg_alpha.it_check_rem = 0
ticket.linbreg_alpha.debug_it_int = 500
ticket.linbreg_alpha.kick.ints = ticket.linbreg_alpha.maxit # dont kick when density is high.
ticket.linbreg_alpha.kick.eval_ints = 2000
ticket.linbreg_alpha.kick.flag = True
ticket.linbreg_alpha.kick.thres = 0.1
ticket.linbreg_alpha.save_obj_int = 2000
ticket.linbreg_alpha.flag_bg_allow_negative = False # force background to not be negative
ticket.linbreg_alpha.e = True
ticket.linbreg_alpha.PyPRIS_iter = 0
ticket.linbreg_alpha.path_0 = '.'
ticket.linbreg_alpha.stopping_loghistpercdelres_thres = -14
ticket.linbreg_alpha.mu = 1e6
ticket.linbreg_alpha.alpha = 1e-6
ticket.linbreg_alpha.auto_mu = True
ticket.linbreg_alpha.auto_bg = True
ticket.bg_scaling_coef = 10


In [12]:
# define a list of mu-fold and bg-folds to sweep through:
muFolds=[100, 200, 300,400]
bgFolds=[2, 3, 4]


# now calculate the packet of tickets for parameter optimization.
for auto_mu_fold in muFolds:
    for auto_bg_fold in bgFolds:
        ticket_new = copy.deepcopy(ticket)
        ticket_new.name="frame"+str(frameN)+'_muFold'+str(auto_mu_fold)+'_bgFold'+str(auto_bg_fold)
        ticket_new.linbreg_alpha.PyPRIS_name = ticket_new.name
        ticket_new.linbreg_alpha.auto_mu_fold = auto_mu_fold;
        ticket_new.linbreg_alpha.auto_bg_fold = auto_bg_fold  # increase number to make the bg compoentn more competitive than random background 'non-specific-emitters'.
                                                    # use a larger number when you think the background component in the calibrated PSF matrix
            # is significantly smaller than tht in the data. 
            # you may consider using a number smaller than 1 when you thin the background component in PSF calibration is big or comparable.
        try:
            if not os.path.exists("{}/{}".format(ticket_new.ticket_pack_folder, ticket_new.name)):
                os.mkdir("{}/{}".format(ticket_new.ticket_pack_folder, ticket_new.name))
        except OSError:
            pass
        with open("{}/{}/Go.pris_ticket".format(ticket_new.ticket_pack_folder, ticket_new.name), "wb") as f:
            pickle.dump(ticket_new, f, pickle.HIGHEST_PROTOCOL)


## then we'll have to switch to terminal :-)
go to the ticket pack folder..

For example this is what I have:




`(base) moti:ticket_pack yi10$ ls
frame0_muFold100_bgFold2	frame0_muFold200_bgFold3	frame0_muFold300_bgFold4
frame0_muFold100_bgFold3	frame0_muFold200_bgFold4	frame0_muFold400_bgFold2
frame0_muFold100_bgFold4	frame0_muFold300_bgFold2	frame0_muFold400_bgFold3
frame0_muFold200_bgFold2	frame0_muFold300_bgFold3	frame0_muFold400_bgFold4`






I tend to think about each folder as a separate basket, that houses a 'ticket' of an 'order'. This order contains specifications of what you want the pris solver to do...



#### you go into a folder, and you see a pris ticket: 

`(base) moti:frame0_muFold100_bgFold2 yi10$ ls
Go.pris_ticket`

#### and copy a pris script into this folder:
`(base) moti:frame0_muFold100_bgFold2 yi10$ echo $ori
/Users/yi10/PyPRIS/tools/pris_moti
(base) moti:frame0_muFold100_bgFold2 yi10$ scp $ori .
(base) moti:frame0_muFold100_bgFold2 yi10$ ls
Go.pris_ticket	pris_moti`


This pris script will be the worker to handle this task (take the order according to the ticket)..... 


#### then you activate the pris environment, and execute the task.




